<a href="https://colab.research.google.com/github/Falconwatch/llm_course/blob/main/HW3/%D0%94%D0%973_done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание ДЗ3.

В семинаре 7 дообучите на задачу предсказания гиперонимов одну декодерную модель (например, GPT2) и одну энкодер-декодерную (например, BART или T5). Сравните полученные метрики с FLAN-T5 из семинара.

**Используйте аналогичные данные и аналогичное разделение на подвыборки** (random_state зафиксирован у меня в семинаре).

За задание можно получить максимум 10 баллов (за каждую из двух моделей 5 баллов)

Разбалловка:
*   **Воспроизводимость и читабельность кода - 3 балла** (все воспроизвелось и все понятно для проверяющего - 3 балла; есть непонятные моменты, но все воспроизвелось - 2 балла; непонятный код и/или воспроизводится с небольшой правкой - 1 балл; непонятный код и/или ничего не воспроизвелось - 0 баллов).
*   **Технический отчет - 2 балла** (приведены результаты сравнения и выводы, почему данная модель лучше/хуже модели из семинара - 2 балла, только результаты сравнения - 1 балл, ничего нет - 0 баллов).


# 1. Информация о сабмите

**Щербаков Игорь Андреевич**

# 2. Технический отчет

***Введите сюда** подробное описание предложенных методов и экспериментов, с помощью которых вы пришли именно к выбору этих методов. НЕ вставляйте код в эту часть. Описание должно состоять минимум из 2-4 абзацев и содержать следующее: тип модели, параметры, как вы выбрали параметры, какие дальнейшие модификации готовых решений и т.д. вы использовали. Сюда можно включить, например, некоторые хитрости вашей предварительной обработки, описание моделей и мотивацию их использования, описание деталей процесса обучения. Если нужно, вставьте сюда графики, математические формулы.*


Пример решения на полный балл для похожей задачи (цифры в таблице приведены информативно):

Method | F1 Average |
--- | --- |
Baseline | 0.447 |
Rubert-tiny2 (unfreeze first layer) | 0.514 |
Rubert-tiny2 (unfreeze all layers) | 0.575 |
Distilbert (unfreeze all layers) | 0.662 |
Distilbert (unfreeze all layers, custom loss) | 0.664 |
Deberta_base | 0.740 |

Выводы:
*   Вывод 1
*   Вывод 2
*   Вывод 3


# 3. *Code*

*Введите сюда весь код, использованный для получения результатов. Добавьте несколько комментариев и подразделов для навигации по вашему решению.*

*В этой части вам предстоит самостоятельно разработать решение задачи и предоставить воспроизводимый код:*
- *Использование Python 3;*
- *Содержит код для установки всех зависимостей;*
- *Содержит код для загрузки всех используемых наборов данных*;
- *Содержит код для воспроизведения ваших результатов (другими словами, если проверяющий загрузит ваш блокнот, он сможет выполнить код по ячейкам и получить результаты эксперимента, как описано в разделе методологии)*.


*В результате ваш код будет оценен по следующим критериям:*
- ***Читаемость**: ваш код должен быть хорошо структурирован, желательно с указанием частей вашего подхода (предварительная обработка, обучение модели, тестирование модели и т. д.).*
- ***Воспроизводимость**: ваш код должен воспроизводиться без ошибок в режиме «Выполнить все» (получение экспериментальной части).*


## 3.1 Импорт библиотек

In [1]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 14.7 MB/s eta 0:00:00


In [2]:
# some imports
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from typing import List, Dict, Union
from transformers import T5Tokenizer, T5ForConditionalGeneration

from tqdm import tqdm
import collections
import re
import os

In [3]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

## 3.2 Загрузка данных

In [4]:
# download the data
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1EQ7KmpLDgG9_bHbt1Mjz4acKjDQWpMza' -O SemEval2018-Task9.zip

!unzip -q SemEval2018-Task9.zip
!rm SemEval2018-Task9.zip

In [5]:
path_x = 'training/data/1A.english.training.data.txt'
path_y = 'training/gold/1A.english.training.gold.txt'

path_x_test = 'test/data/1A.english.test.data.txt'
path_y_test = 'test/gold/1A.english.test.gold.txt'

In [6]:
def read_Y(path):
    with open(path) as input:
        Y = [line.rstrip('\n').split('\t') for line in input]
    return Y

In [7]:
df = pd.read_csv(path_x, sep='\t', header= None, names=['hyponym', 'property'])
df_test = pd.read_csv(path_x_test, sep='\t', header= None, names=['hyponym', 'property'])

df.insert(2, 'hypernyms', read_Y(path_y))
df_test.insert(2, 'hypernyms', read_Y(path_y_test))

df['hypernyms'] = [', '.join(map(str, l)) for l in df['hypernyms']]
df_test['hypernyms'] = [', '.join(map(str, l)) for l in df_test['hypernyms']]

df_test.head()

,hyponym,property,hypernyms
0,maliciousness,Concept,"malevolence, distaste, hatred, hate, malignity"
1,buckler,Concept,body armor
2,spelunker,Concept,"exploration, adventurer, explorer"
3,quo warranto,Concept,"proceedings, legal proceedings, proceeding, du..."
4,Jeff Francis,Entity,"thrower, baseball player, jock, person"


In [8]:
def generate_input_prompt(dataframe):
    _input = []
    for hyponym in dataframe.hyponym:
        input_text = f"hyponym: {hyponym}, predict hypernyms:"
        _input.append(input_text)
    return _input

df.insert(3, 'hyponyms_input', generate_input_prompt(df))
df_test.insert(3, 'hyponyms_input', generate_input_prompt(df_test))
df.head()

,hyponym,property,hypernyms,hyponyms_input
0,blackfly,Concept,"homopterous insect, insect","hyponym: blackfly, predict hypernyms:"
1,Turonian,Entity,"technical specification, geologic timescale, p...","hyponym: Turonian, predict hypernyms:"
2,abhorrence,Concept,"distaste, hatred, hate, disgust","hyponym: abhorrence, predict hypernyms:"
3,tropical storm,Concept,"atmosphere, windstorm, violent storm, air curr...","hyponym: tropical storm, predict hypernyms:"
4,militarization,Concept,social control,"hyponym: militarization, predict hypernyms:"


In [9]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx <= len(self.x['input_ids']), (idx, len(self.x['input_ids']))
        item = {key: val[idx] for key, val in self.x.items()}
        item['labels'] = self.y['input_ids'][idx]
        return item

    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n

## 3.3. Модель

In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import gc

def prepair_datasets(tokenizer, train_df, test_df):
  train_dataset = PairsDataset(tokenizer(train_df['hyponyms_input'].tolist(), padding="max_length",  max_length=50, truncation=True),
                              tokenizer(train_df['hypernyms'].tolist(),  padding="max_length",  max_length=50, truncation=True))
  test_dataset = PairsDataset(tokenizer(test_df['hyponyms_input'].tolist(),  padding="max_length",  max_length=50, truncation=True),
                            tokenizer(test_df['hypernyms'].tolist(),  padding="max_length",  max_length=50, truncation=True))

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
  return train_dataset, test_dataset, data_collator

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
EPOCHS=1
ACTIVATE_SAVING = True

### 3.3.1 Baseline model

#### 3.3.1.1 Обучение

In [12]:
baseline_model_name = "flan-t5-base-finetuned2"

In [13]:
# model training
tokenizer_t5 = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_t5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").cuda()
model_t5.resize_token_embeddings(len(tokenizer_t5))

train_dataset_t5, test_dataset_t5, data_collator_t5 = prepair_datasets(tokenizer_t5, df, df_test)

args_t5 = TrainingArguments(output_dir="bart-base-finetuned",
                          num_train_epochs=EPOCHS,
                          per_device_train_batch_size=32,
                          save_steps=10000)

trainer_t5 = Trainer(
    model = model_t5,
    args = args_t5,
    train_dataset = train_dataset_t5,
    eval_dataset = test_dataset_t5,
    tokenizer = tokenizer_t5
)
#cleanup()
trainer_t5.train()

#сохраняем
print("Сохраняем в", baseline_model_name)
trainer_t5.save_model(baseline_model_name)
cleanup()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step,Training Loss


Сохраняем в flan-t5-base-finetuned2


#### 3.3.1.2 Применение

In [ ]:
p = re.compile(r"[,\s]")
with open("./res_"+baseline_model_name, "w", encoding='utf-8') as w:
    for i_2, j_2 in tqdm(zip(df_test['hyponyms_input'].tolist(),
                             df_test['hypernyms'].tolist()),
                         total=1500):
        input_ids = tokenizer_t5.encode(i_2, return_tensors="pt")
        outputs = model_t5.generate(input_ids.to('cuda'),
                                    no_repeat_ngram_size=2, max_new_tokens=2048,
                                      num_return_sequences=5, early_stopping=True, do_sample=True, num_beams=5)
        #print(outputs)
        decoded = [tokenizer_t5.decode(output, skip_special_tokens=True) for output in outputs]
        decoded = [j for i in decoded for j in i.split(", ")]
        decoded = [i for i in decoded if i and i!='.' and i!=',']
        decoded = [i[0].strip() for i in list(collections.Counter(decoded).most_common())]
        w.write('\t'.join(decoded)+'\n')

 10%|█         | 151/1500 [00:47<10:14,  2.19it/s]

In [ ]:
!python task9-scorer.py test/gold/1A.english.test.gold.txt res_flan-t5-base-finetuned

### 3.3.2 GPT2

### 3.3.3 BART

## 3.4. Применение модели

In [ ]:
# model inference